# _Cassava Leaf Disease Detection_



## _Import Libraries & Load Data_

In [ ]:
#system related imports
import os
import sys
assert sys.version_info >= (3,5)
#data manipulation
import pandas as pd
import numpy as np
#visualization imports
import matplotlib.pyplot as plt
from matplotlib.image import imread
import seaborn as sns
%matplotlib inline
# consistent plot size
from pylab import rcParams
rcParams['figure.figsize'] = 15,7
rcParams['axes.labelsize'] = 14
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['axes.titlesize'] = 16
#ignore warnings
import warnings
warnings.filterwarnings(action='ignore',message='')
#deep learning framework
import tensorflow as tf

## _Load the data_

In [ ]:
'''read the train csv file'''
train =  pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
train.head(10)

In [ ]:
'''read the disease label into a dataframe'''
data = {"0":"Cassava Bacterial Blight (CBB)",
        "1":"Cassava Brown Streak Disease (CBSD)",
        "2":"Cassava Green Mottle (CGM)",
        "3":"Cassava Mosaic Disease (CMD)",
        "4":"Healthy"}
disease_label = pd.DataFrame(data=data.values(),index=data.keys(),columns=['Disease_Name'])

In [ ]:
disease_label

In [ ]:
train_path = '../input/cassava-leaf-disease-classification/train_images'
test_path = '../input/cassava-leaf-disease-classification/test_images'

In [ ]:
'''Join the train path'''
def image_path(image):
    return os.path.join(train_path,image)

train['image_id'] = train['image_id'].apply(image_path)

In [ ]:
train.head(3)

In [ ]:
train.info()

In [ ]:
train['label'] = train['label'].astype('str')

In [ ]:
str(disease_label.iloc[3])

In [ ]:
'''view one of the images in the train set randomly'''
index = np.random.randint(0,len(train))

plt.imshow(imread(train['image_id'][index]))
plt.title('Cassava Leaf')

plt.show()

In [ ]:
'''check the count of the various disease types'''
sns.countplot(train['label'])
plt.title('Count of the various disease types in Cassava leaves')
plt.grid()
plt.show()

- <b> _The Cassava Mosaic Disease or CMD is most prevalent as per the dataset_ </b>

In [ ]:
'''display the images of the same label'''
n_rows = 3
n_cols = 4

for row in range(n_rows):
    for col in range(n_cols):
        index = n_cols * row + col
        plt.subplot(n_rows,n_cols,index+1)
        
        leaf_img = np.random.randint(0,len(train))
        image_ = imread(train.iloc[leaf_img]['image_id'])
        
        '''display the image'''
        plt.imshow(image_,cmap='binary',interpolation='nearest')     
        

# _Data Preparation_ 

In [ ]:
'''image data augmentation'''
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=0.5,
                                                                 horizontal_flip=True,
                                                                 vertical_flip=True,
                                                                 brightness_range=[0.1,0.3],
                                                                 zoom_range=[0.1,0.5],
                                                                 fill_mode='nearest',
                                                                 preprocessing_function=tf.keras.applications.xception.preprocess_input,
                                                                 validation_split=0.2,
                                                                  rescale = 1./255,
                                                                  dtype='float32')

In [ ]:
train_generator = image_generator.flow_from_dataframe(dataframe=train,
                                                      directory=None,
                                                      x_col='image_id',
                                                      y_col='label',
                                                      subset='training',
                                                      color_mode='rgb',
                                                      batch_size=32,
                                                      seed=42,
                                                      shuffle=True,
                                                      class_mode='categorical',
                                                      target_size=(224,224))

In [ ]:
'''image generator on the validation data. 
   note: shuffle is set to false in this case'''

validation_generator = image_generator.flow_from_dataframe(dataframe=train,
                                                           directory=None,
                                                           x_col='image_id',
                                                           y_col='label',
                                                           subset='validation',
                                                           color_mode='rgb',
                                                           batch_size=32,
                                                           seed=42,
                                                           shuffle=False,
                                                           class_mode='categorical',
                                                           target_size=(224,224))

# _Functional API Convolutional Neural Network Model_

In [ ]:
def cnn_model_pretrain(xception=True,shape=(224,224,3),filters=8,kernel=4,stride=1,pad='same',unit=1024,activation=tf.nn.relu,
              regularizer=tf.keras.regularizers.L1L2(l1=0.01,l2=0.01)):
    
    '''function to define the cnn model architecture with or without transfer learning'''
    
    input_img = tf.keras.Input(shape=shape)
    
    base_model = tf.keras.applications.xception.Xception(include_top=False,input_shape=shape,weights='imagenet')
    for layer in base_model.layers:
        layer.trainable = False
    
    if xception:
        pretrain = base_model(input_img)
    else:
        pretrain =  input_img
        
    '''One layer of Conv2D, Activation and Pooling layer'''
    Z1 = tf.keras.layers.Conv2D(filters=filters,kernel_size=(kernel,kernel),strides=stride,padding=pad)(pretrain)
    A1 = tf.keras.layers.Activation(activation)(Z1)
    P1 = tf.keras.layers.MaxPool2D(pool_size=(filters,filters),strides=stride*8,padding=pad)(A1)
    
    '''Second layer of Conv2D, Activation and Pooling layer'''
    Z2 = tf.keras.layers.Conv2D(filters=filters*2,kernel_size=(int(kernel/2),int(kernel/2)),strides=stride,
                               padding=pad) (P1)
    A2 = tf.keras.layers.Activation(activation)(Z2)
    P2 = tf.keras.layers.MaxPool2D(pool_size=int(filters/2),strides=stride*4,padding=pad)(A2)
    
    '''Flatten the output to feed into the dense layer'''
    F = tf.keras.layers.Flatten()(P2)
    
    '''Dense Layers'''
    D1 = tf.keras.layers.Dense(units=unit,activation=activation)(F)
    D2 = tf.keras.layers.Dense(units=unit/2,activation=activation,kernel_regularizer=regularizer)(D1)
    D3 = tf.keras.layers.Dense(units=unit/4,activation=activation,kernel_regularizer=regularizer)(D2)
    
    '''Output layer'''
    outputs = tf.keras.layers.Dense(units=5,activation=tf.nn.softmax)(D3)
    
    '''instantiate the model'''
    model = tf.keras.Model(inputs=input_img,outputs=outputs)
    return model  

In [ ]:
regularizer = tf.keras.regularizers.L2(l2=0.005)
shape = (224,224,3)

conv_model = cnn_model_pretrain(xception=True,shape=shape,filters=8,kernel=4,stride=1,pad='same',unit=256,activation=tf.nn.relu,
                      regularizer=regularizer)

'''compile the model'''
conv_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.summary()

In [ ]:
'''fit the model'''
history = conv_model.fit(train_generator,epochs=5,validation_data=validation_generator,batch_size=32)

In [ ]:
results = pd.DataFrame(history.history)
results[['loss','val_loss']].plot()
plt.title('Model Performance with pretrained')
plt.show()

In [ ]:
'''without pretrained xception module'''

conv_model = cnn_model_pretrain(xception=False,shape=shape,filters=8,kernel=4,stride=1,pad='same',unit=256,activation=tf.nn.relu,
                      regularizer=regularizer)

'''compile the model'''
conv_model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
conv_model.summary()

In [ ]:
'''fit the model'''
history = conv_model.fit(train_generator,epochs=5,validation_data=validation_generator,batch_size=32)

In [ ]:
results = pd.DataFrame(history.history)
results[['loss','val_loss']].plot()
plt.title('Model Performance without pretrained network')
plt.show()

- <b> _Train for longer to see the variation_ </b>
- <b> _Use other pretrained models like VGG16 and Reset50. Accuracy increases by atleast 6%_ </b> 
- <b> _Use GlobalAveragePooling instead of MaxPooling_ </b> 


## _Upvote if you liked the notebook. Thank you !_